In [67]:
import os
import pandas as pd
import requests
import sys
import json
from collections import defaultdict
from __init__ import get_base_path
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv(get_base_path() + '/airbnb.env')

# Access the API key from the environment variable
data_dir = get_base_path() + os.getenv('data_dir')

In [68]:
df = pd.read_csv(data_dir + 'raw/' + 'test_google_response.csv', index_col=0)
df.head()

,id,last_scraped,latitude,longitude,accommodates,bedrooms,price,minimum_nights,instant_bookable,calculated_host_listings_count_entire_homes,...,room_type_Entire home/apt,room_type_Hotel room,room_type_Private room,room_type_Shared room,neighbourhood_group_cleansed_Bronx,neighbourhood_group_cleansed_Brooklyn,neighbourhood_group_cleansed_Manhattan,neighbourhood_group_cleansed_Queens,neighbourhood_group_cleansed_Staten Island,geocoding_api_response
38803,9357,2024-08-05,40.767240,-73.986640,2,1.0,175.0,30,0,1,...,1,0,0,0,0,0,1,0,0,{'address_components': [{'long_name': '859-861...
38805,10452,2024-08-05,40.682940,-73.956820,2,2.0,90.0,30,0,0,...,0,0,1,0,0,1,0,0,0,"{'address_components': [{'long_name': '108', '..."
38820,26785,2024-08-05,40.668600,-73.987230,2,3.0,90.0,30,1,3,...,1,0,0,0,0,1,0,0,0,"{'address_components': [{'long_name': '360', '..."
38821,15341,2024-08-05,40.721359,-73.993668,3,2.0,228.0,30,0,1,...,1,0,0,0,0,0,1,0,0,"{'address_components': [{'long_name': '209', '..."
38822,15385,2024-08-05,40.712110,-73.963970,1,1.0,98.0,30,0,0,...,0,0,1,0,0,1,0,0,0,"{'address_components': [{'long_name': '98a', '..."


In [69]:
# Function to extract individual address components from Google API response get_address()
def get_address_component(json, component):

    result = [item['long_name'] for item in json['address_components'] if item['types'] == [component]]
    if len(result) > 0:
        return result[0]
    else:
        return None

In [71]:
# string to json
df['cleaned'] = df['geocoding_api_response'].str.replace("\'",'\"').apply(lambda x: json.loads(x))

# process address components
df['house_num'] = df['cleaned'].apply(lambda x: get_address_component(x, 'street_number').upper())
df['street_name'] = df['cleaned'].apply(lambda x: get_address_component(x, 'route').upper())
df['zip_code'] = df['cleaned'].apply(lambda x: get_address_component(x, 'postal_code'))
df['full_address'] = df['cleaned'].apply(lambda x: x['formatted_address'])
df

,id,last_scraped,latitude,longitude,accommodates,bedrooms,price,minimum_nights,instant_bookable,calculated_host_listings_count_entire_homes,...,neighbourhood_group_cleansed_Brooklyn,neighbourhood_group_cleansed_Manhattan,neighbourhood_group_cleansed_Queens,neighbourhood_group_cleansed_Staten Island,geocoding_api_response,cleaned,house_num,street_name,zip_code,full_address
38803,9357,2024-08-05,40.767240,-73.986640,2,1.0,175.0,30,0,1,...,0,1,0,0,{'address_components': [{'long_name': '859-861...,{'address_components': [{'long_name': '859-861...,859-861,9TH AVENUE,10019,"859-861 9th Ave, New York, NY 10019, USA"
38805,10452,2024-08-05,40.682940,-73.956820,2,2.0,90.0,30,0,0,...,1,0,0,0,"{'address_components': [{'long_name': '108', '...","{'address_components': [{'long_name': '108', '...",108,PUTNAM AVENUE,11238,"108 Putnam Ave, Brooklyn, NY 11238, USA"
38820,26785,2024-08-05,40.668600,-73.987230,2,3.0,90.0,30,1,3,...,1,0,0,0,"{'address_components': [{'long_name': '360', '...","{'address_components': [{'long_name': '360', '...",360,10TH STREET,11215,"360 10th St, Brooklyn, NY 11215, USA"
38821,15341,2024-08-05,40.721359,-73.993668,3,2.0,228.0,30,0,1,...,0,1,0,0,"{'address_components': [{'long_name': '209', '...","{'address_components': [{'long_name': '209', '...",209,BOWERY,10002,"209 Bowery, New York, NY 10002, USA"
38822,15385,2024-08-05,40.712110,-73.963970,1,1.0,98.0,30,0,0,...,1,0,0,0,"{'address_components': [{'long_name': '98a', '...","{'address_components': [{'long_name': '98a', '...",98A,SOUTH 4TH STREET,11249,"98a S 4th St, Brooklyn, NY 11249, USA"
38827,28907,2024-08-05,40.778270,-73.946690,4,2.0,150.0,30,0,1,...,0,1,0,0,"{'address_components': [{'long_name': '427A', ...","{'address_components': [{'long_name': '427A', ...",427A,EAST 89TH STREET,10128,"427A E 89th St, New York, NY 10128, USA"
38830,68735,2024-08-05,40.788460,-73.974590,2,1.0,225.0,30,0,1,...,0,1,0,0,"{'address_components': [{'long_name': '560', '...","{'address_components': [{'long_name': '560', '...",560,AMSTERDAM AVENUE,10024,"560 Amsterdam Ave, New York, NY 10024, USA"
